In [113]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from collections import Counter


In [114]:
converter = lambda x: list(map(lambda y: y.strip("'"), x.strip("[]").split(", ")))
df = pd.read_csv('data.csv', converters={"Email" : converter, "Lemmatized" : converter}, index_col=0)
df

,Email,Lemmatized,Spam
0,"[christmas, tree, farm, pictures]","[christmas, tree, farm, picture]",False
1,"[calpine, daily, gas, nomination, calpine, dai...","[calpine, daily, gas, nomination, calpine, dai...",False
2,"[mcmullen, gas, jackie, since, inlet, river, p...","[mcmullen, gas, jackie, since, inlet, river, p...",False
3,"[meter, 1517, jan, 1999, george, need, followi...","[meter, 1517, jan, 1999, george, need, follow,...",False
4,"[king, ranch, two, fields, gas, difficulty, un...","[king, ranch, two, field, gas, difficulty, uni...",False
...,...,...,...
24927,"[need, vics, brand, name, meds, vics, vals, xa...","[need, vics, brand, name, med, vics, vals, xan...",True
24928,"[iso, 8859, good, news, edaliss, val, edumm, e...","[iso, 8859, good, news, edaliss, val, edumm, e...",True
24929,"[prescript, medicines, special, precise, put, ...","[prescript, medicine, special, precise, put, b...",True
24930,"[next, generation, online, pharmacy, ready, ro...","[next, generation, online, pharmacy, ready, ro...",True


The dataframe has ham and spam saperated. Thus shuffle rows and reset the index.

In [115]:
df = df.sample(frac=1).reset_index(drop=True)
df

,Email,Lemmatized,Spam
0,"[enron, hpl, actuals, january, 2001, teco, tap...","[enron, hpl, actuals, january, 2001, teco, tap...",False
1,"[movie, lovers, love, kqo, introducing, 2005, ...","[movie, lover, love, kqo, introducing, 2005, d...",True
2,"[great, nnews, hello, welcome, medzonline, gro...","[great, nnews, hello, welcome, medzonline, gro...",True
3,"[meoh, upset, meoh, plant, began, problems, ju...","[meoh, upset, meoh, plant, begin, problem, jun...",False
4,"[toronto, pharmacy, prescriptions, cptcy, utqg...","[toronto, pharmacy, prescription, cptcy, utqgr...",True
...,...,...,...
24927,"[start, date, hourahead, hour, start, date, ho...","[start, date, hourahead, hour, start, date, ho...",False
24928,"[california, litigation, team, starting, thurs...","[california, litigation, team, start, thursday...",False
24929,"[good, news, men, problems, small, penis, poor...","[good, news, men, problem, small, penis, poor,...",True
24930,"[save, money, getting, oem, software, need, so...","[save, money, get, oem, software, need, softwa...",True


In [116]:
k_fold_indexs = pd.DataFrame(columns = ['train_index','test_index'])

from sklearn.model_selection import KFold  
import numpy as np  
X = df[['Email','Lemmatized']] 
y = df['Spam'] 
kf = KFold(n_splits=10,shuffle=True) 
i = 0
for train_index , test_index in kf.split(X):  
    k_fold_indexs.at[i,'train_index'] = train_index
    k_fold_indexs.at[i,'test_index'] = test_index
    i+=1
k_fold_indexs

,train_index,test_index
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[32, 39, 66, 69, 75, 84, 94, 98, 108, 118, 136..."
1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14,...","[11, 17, 22, 47, 54, 71, 77, 90, 97, 99, 104, ..."
2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[48, 56, 96, 116, 127, 129, 134, 145, 151, 155..."
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15...","[0, 10, 30, 38, 42, 51, 65, 72, 79, 95, 100, 1..."
4,"[0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...","[1, 7, 20, 27, 35, 53, 62, 76, 81, 86, 93, 103..."
5,"[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...","[4, 14, 18, 23, 52, 67, 78, 101, 117, 121, 122..."
6,"[0, 1, 4, 5, 6, 7, 9, 10, 11, 12, 14, 16, 17, ...","[2, 3, 8, 13, 15, 24, 33, 40, 43, 44, 45, 58, ..."
7,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14...","[6, 28, 36, 49, 50, 60, 85, 91, 107, 144, 154,..."
8,"[0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 1...","[5, 9, 16, 19, 25, 26, 46, 55, 57, 61, 64, 68,..."
9,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...","[12, 21, 29, 31, 34, 37, 41, 59, 74, 82, 113, ..."


In [161]:
# train0 = df.loc[list(k_fold_indexs['train_index'][0])]
# test0 = df.loc[list(k_fold_indexs['test_index'][0])]

train0 = df.sample(frac=0.1)

In [118]:
# train_ham0 = train0[~train0.Spam]
# train_spam0 = train0[train0.Spam]

In [163]:
train0

,Email,Lemmatized,Spam
19415,"[defeat, spam, porn, clear, inbox, spam, porn,...","[defeat, spam, porn, clear, inbox, spam, porn,...",True
11477,"[valero, gas, marketing, meter, 8018, sitara, ...","[valero, gas, marketing, meter, 8018, sitara, ...",False
10707,"[kenney, song, sat, apr, 2004, 0600, sir, mada...","[kenney, song, sit, apr, 2004, 0600, sir, mada...",True
12493,"[reduc, mor, tgage, hundreds, monthly, preferr...","[reduc, mor, tgage, hundred, monthly, preferre...",True
24536,"[ena, origination, track, recent, discussions,...","[ena, origination, track, recent, discussion, ...",False
...,...,...,...
9460,"[samurai, stock, info, little, continue, impor...","[samurai, stock, info, little, continue, impor...",True
20239,"[heartwarmer, imagine, welcome, heartwarmers, ...","[heartwarmer, imagine, welcome, heartwarmers, ...",True
13398,"[professional, review, met, gas, uproot, duck,...","[professional, review, meet, gas, uproot, duck...",True
13005,"[ever, bought, drugs, online, regain, confiden...","[ever, buy, drug, online, regain, confidence, ...",True


In [164]:
# train_spam0

In [165]:
train_words = train0.Lemmatized.values.tolist()

In [166]:
dictionary = gensim.corpora.Dictionary(train_words)
corpus = [dictionary.doc2bow(text) for text in train_words]

In [167]:
tfidf=gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [168]:
# num_topics=5
# lda = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,passes=100) 

# for topic in lda.print_topics(num_words=num_topics,num_topics=num_topics):
#     print(topic)

In [169]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 2),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 3),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1)],
 [(30, 2),
  (76, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 2),
  (85, 1),
  (86, 2),
  (87, 1),
  (88, 1),
  (89, 2

In [126]:
# model = TfidfModel(corpus)  # fit model
# dic = dict(corpus[0])  # apply model to the first corpus document

In [170]:
len(corpus) == train0.shape[0]

True

In [172]:
dic = dict(corpus[0])
dic

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 2,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 1,
 68: 3,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 1}

In [129]:
corpus[3]

[(86, 1),
 (87, 2),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 2),
 (99, 1),
 (100, 2),
 (101, 3),
 (102, 1),
 (103, 1),
 (104, 1),
 (105, 1),
 (106, 1)]

In [173]:
tfidfs = pd.DataFrame(columns = list(range(train0.shape[0])))
# for i in list(train0.index)[:100]:
for i in range(len(corpus)):
    dic = dict(corpus[i])
    for key in dic.keys():
        tfidfs.at[i,key] = dic[key]
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
3

1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098


In [174]:
tfidfs.index = list(train0.index)

In [175]:
tfidfs

,0,1,2,3,4,5,6,7,8,9,...,35507,35508,35509,35510,35511,35512,35513,35514,35515,35516
19415,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
tfidfs['Spam'] = train0.Spam

In [177]:
tfidfs

,0,1,2,3,4,5,6,7,8,9,...,35508,35509,35510,35511,35512,35513,35514,35515,35516,Spam
19415,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
11477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
10707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
12493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
20239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
13398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
13005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [236]:
tfidfs = tfidfs.fillna(0)
# tfidfs = tfidfs.reset_index()
# tfidfs.drop('level_0',axis = 1)
tfidfs

,index,0,1,2,3,4,5,6,7,8,...,35508,35509,35510,35511,35512,35513,35514,35515,35516,Spam
0,19415,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,11477,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,10707,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,12493,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,24536,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,9460,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2489,20239,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2490,13398,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2491,13005,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [237]:
from sklearn import svm
from sklearn.preprocessing import scale

# X = tfidfs.iloc[:,:-1].to_frame()
# Y = tfidfs.iloc[:,-1].to_frame()

X = tfidfs.iloc[:,1:-1]
Y = tfidfs.iloc[:,-1]

clf = svm.SVC()
clf.fit(X, y)


ValueError: Found input variables with inconsistent numbers of samples: [2493, 2]

In [201]:
X.shape

(2493, 35517)

In [202]:
Y.shape

(2493, 1)

In [ ]:
# meaningless stuff

In [81]:
# tfidfs.fillna(0)

[(0, 0.0831173037221004),
 (1, 0.09504599178894553),
 (2, 0.21233354753258937),
 (3, 0.06190486454238464),
 (4, 0.24437164459580676),
 (5, 0.14843944968347364),
 (6, 0.16898013566739992),
 (7, 0.07485891984055598),
 (8, 0.16479523366657564),
 (9, 0.22126344087807157),
 (10, 0.31098300979953336),
 (11, 0.22514505950726044),
 (12, 0.19423219666087913),
 (13, 0.10249426226932923),
 (14, 0.0908752085245007),
 (15, 0.2054069841322857),
 (16, 0.14164525680602222),
 (17, 0.05554862672770087),
 (18, 0.09728275723533494),
 (19, 0.07297345074935374),
 (20, 0.05833699329240777),
 (21, 0.13984154710230182),
 (22, 0.12862022163032077),
 (23, 0.08246001401194246),
 (24, 0.14191170342543027),
 (25, 0.03619772812856633),
 (26, 0.13183262604414908),
 (27, 0.14500429743421966),
 (28, 0.07341209451449741),
 (29, 0.23136482572748407),
 (30, 0.31098300979953336),
 (31, 0.12002453062035272),
 (32, 0.17028731157026922),
 (33, 0.11703850706479349),
 (34, 0.09528339899262096),
 (35, 0.10439254696850964),
 (36,

In [63]:
dic

{25: 0.05052795015228481,
 42: 0.042680493828282075,
 43: 0.03973311161419778,
 57: 0.035637606005308374,
 165: 0.04915208934377423,
 212: 0.05593610808138067,
 273: 0.06875735059913125,
 355: 0.24486176910008162,
 652: 0.09331342210814338,
 678: 0.12588823817632053,
 770: 0.1405239095878567,
 802: 0.07342507094540876,
 834: 0.04553290301547659,
 842: 0.06717820355363918,
 905: 0.06953243273531294,
 933: 0.0536139579400161,
 984: 0.04988301797652206,
 1028: 0.1403757743456776,
 1095: 0.17206753353825693,
 1122: 0.10237045356657927,
 1243: 0.08232415486076897,
 1273: 0.0768268053731532,
 1281: 0.07316901698308946,
 1301: 0.0699671916337989,
 1536: 0.4014067774830036,
 1555: 0.0686652579713349,
 1821: 0.09290739647131643,
 2131: 0.08316005631788576,
 2139: 0.10314625669575793,
 2251: 0.2701301381406475,
 2678: 0.07771539003048464,
 2915: 0.21757045409703296,
 2960: 0.12105604798280396,
 2998: 0.15566774031744476,
 3460: 0.06857355515476979,
 3778: 0.10314625669575793,
 5511: 0.1124443729

In [69]:
i = 1
for key in dic.keys():
    tfidfs.at[i,key] = dic[key]
tfidfs


,0,1,2,3,4,5,6,7,8,9,...,22428,22429,22430,22431,22432,22433,22434,22435,22436,22437
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
dic.keys()

dict_keys([25, 42, 43, 57, 165, 212, 273, 355, 652, 678, 770, 802, 834, 842, 905, 933, 984, 1028, 1095, 1122, 1243, 1273, 1281, 1301, 1536, 1555, 1821, 2131, 2139, 2251, 2678, 2915, 2960, 2998, 3460, 3778, 5511, 5622, 5879, 6092, 6934, 6935, 6936, 6937, 6938, 6939, 6940, 6941, 6942, 6943])

In [68]:
dic[25]

0.05052795015228481

In [79]:
tfidfs.at[1,6943]

0.1322955900036992

In [38]:
ham_index = list(train0[~train0.Spam].index)
spam_index = list(train0[train0.Spam].index)

In [50]:
ham_tfidf = {}
for i in ham_index[:100]:
    dic = dict(model[corpus[i]])
    ham_tfidf = dict(Counter(ham_tfidf)+Counter(dic))


spam_tfidf = {}
for i in spam_index[:100]:
    dic = dict(model[corpus[i]])
    spam_tfidf = dict(Counter(spam_tfidf)+Counter(dic))


In [51]:
ham_tfidf

{0: 0.6676052635502641,
 1: 0.26240553280409074,
 2: 0.21233354753258937,
 3: 0.4895927167448096,
 4: 0.24437164459580676,
 5: 0.14843944968347364,
 6: 0.16898013566739992,
 7: 0.3203860911724429,
 8: 0.16479523366657564,
 9: 0.22126344087807157,
 10: 0.31098300979953336,
 11: 0.5239495460083214,
 12: 0.4166053709394215,
 13: 0.5395001444495275,
 14: 0.29341326404976037,
 15: 0.2054069841322857,
 16: 0.9907323726675324,
 17: 0.9245215299967355,
 18: 0.3317170709970714,
 19: 0.8456180820088959,
 20: 0.7568344325196524,
 21: 0.5920599017046219,
 22: 0.233135065659003,
 23: 0.231752029593639,
 24: 0.20922493192735203,
 25: 1.0087621964007718,
 26: 0.2827655817069942,
 27: 0.14500429743421966,
 28: 0.6332521167578362,
 29: 0.23136482572748407,
 30: 0.31098300979953336,
 31: 1.0047201752699437,
 32: 0.341003526175074,
 33: 0.35686353364692935,
 34: 0.14145596975390562,
 35: 0.4549167853766233,
 36: 0.8398402849788108,
 37: 0.19282100635528263,
 38: 0.5760790329022825,
 39: 0.452187626446398

In [52]:
spam_tfidf

{17: 0.7383448927993512,
 47: 0.33378676806632296,
 58: 0.3136344614242944,
 71: 0.7508538547765606,
 75: 0.15884776739490492,
 76: 0.25799681041764716,
 77: 0.1835639253905689,
 78: 0.25799681041764716,
 79: 0.19624618520496215,
 80: 0.18987088267652963,
 81: 0.21171274442664767,
 82: 0.26654369307433456,
 83: 0.37141733128174703,
 84: 0.24605386463174758,
 85: 0.25819659456852306,
 86: 0.10591764173257148,
 87: 0.31024716677720676,
 88: 0.5937049587953613,
 89: 0.5911255014694223,
 90: 0.13514208794700935,
 91: 0.12079328522174684,
 92: 0.5526310527262022,
 93: 0.08319615241591287,
 94: 0.7271163800961863,
 95: 0.22092366816208997,
 96: 0.40785073827933455,
 97: 0.765159258092935,
 98: 0.31724232282871245,
 99: 0.3105116174881375,
 100: 0.6416752294564025,
 101: 0.15343223773340817,
 102: 0.2569058364242007,
 103: 0.09611971894289024,
 104: 0.086872355310285,
 105: 0.10819450039423532,
 106: 0.7833611866604824,
 107: 0.694958591494852,
 108: 0.2920125597448385,
 109: 0.13814797273179

In [53]:
def avedic(x, length):
    y = {}
    for key in x:
        y[key]=x[key]/length
    return y

In [54]:
ham_tfidf_freq = avedic(ham_tfidf, len(ham_index))
spam_tfidf_freq = avedic(spam_tfidf, len(spam_index))

In [55]:
spam_tfidf_freq

{17: 5.374862726937113e-05,
 47: 2.4298374322364632e-05,
 58: 2.2831365030522995e-05,
 71: 5.4659230892957754e-05,
 75: 1.1563497662874348e-05,
 76: 1.8781161128168244e-05,
 77: 1.3362737525702038e-05,
 78: 1.8781161128168244e-05,
 79: 1.428595655564986e-05,
 80: 1.3821859407187132e-05,
 81: 1.541186171847184e-05,
 82: 1.9403340836742707e-05,
 83: 2.703773249484946e-05,
 84: 1.7911761274786894e-05,
 85: 1.879570463482005e-05,
 86: 7.710391041171397e-06,
 87: 2.2584783197001294e-05,
 88: 4.3219404440224306e-05,
 89: 4.3031630011605326e-05,
 90: 9.837816695567398e-06,
 91: 8.793279844343513e-06,
 92: 4.022938434346671e-05,
 93: 6.0563552752357045e-06,
 94: 5.293123535678724e-05,
 95: 1.6082381026577124e-05,
 96: 2.9689942365824748e-05,
 97: 5.5700608436553465e-05,
 98: 2.309400326335535e-05,
 99: 2.2604034176904527e-05,
 100: 4.671145297054688e-05,
 101: 1.116926823421476e-05,
 102: 1.8701742478284973e-05,
 103: 6.997140492312022e-06,
 104: 6.3239685018770475e-06,
 105: 7.876137467732061

In [56]:
ham_tfidf_freq

{0: 7.672741794624342e-05,
 1: 3.0158089047706097e-05,
 2: 2.4403349906055553e-05,
 3: 5.6268557262936395e-05,
 4: 2.8085466566579333e-05,
 5: 1.7060044786056045e-05,
 6: 1.942077182707734e-05,
 7: 3.6821755105441084e-05,
 8: 1.8939803892262456e-05,
 9: 2.5429656462253944e-05,
 10: 3.5741065371742716e-05,
 11: 6.021716423495247e-05,
 12: 4.788017135265159e-05,
 13: 6.200438391558758e-05,
 14: 3.372178646704521e-05,
 15: 2.3607284695125353e-05,
 16: 0.000113864196375995,
 17: 0.00010625462935257276,
 18: 3.812401689427323e-05,
 19: 9.718630985046499e-05,
 20: 8.69824655234631e-05,
 21: 6.804504099581909e-05,
 22: 2.6794054207447763e-05,
 23: 2.6635102815037237e-05,
 24: 2.4046078833163088e-05,
 25: 0.00011593635172977494,
 26: 3.249805559211518e-05,
 27: 1.6665245079211547e-05,
 28: 7.277923419811931e-05,
 29: 2.6590601738591434e-05,
 30: 3.5741065371742716e-05,
 31: 0.00011547180499597101,
 32: 3.9191302858875306e-05,
 33: 4.101408270853113e-05,
 34: 1.6257438197207863e-05,
 35: 5.2283

In [58]:
test_words = train0.Lemmatized.values.tolist()

dictionary = gensim.corpora.Dictionary(train_words)
corpus = [dictionary.doc2bow(text) for text in train_words]

tfidf=gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

for i in list(test0.index):
    test0.iloc[i,'Lemmatized']
    pass

,Email,Lemmatized,Spam
1,"[chocolate, peanut, kind, like, choose, chocol...","[chocolate, peanut, kind, like, choose, chocol...",True
10,"[password, fast]","[password, fast]",False
12,"[max, nothing, seems, recorded, alledged, opti...","[max, nothing, seem, record, alledged, option,...",False
28,"[great, article, material, lead, divorce, wire...","[great, article, material, lead, divorce, wire...",True
42,"[fwd, maces, vicodin, paneled, roper, blow, cr...","[fwd, mace, vicodin, panel, roper, blow, crop,...",True
...,...,...,...
24883,"[owe, 318551, hello, thank, ort, gage, applica...","[owe, 318551, hello, thank, ort, gage, applica...",True
24908,"[india, dear, bill, currently, helping, milind...","[india, dear, bill, currently, help, milind, l...",False
24910,"[attention, sma, stock, players, santiago, iog...","[attention, sma, stock, player, santiago, iogn...",True
24920,"[need, sales, leads, help, 1000, leads, leads,...","[need, sale, lead, help, 1000, lead, lead, 278...",True


In [28]:
dict(vector)

{0: 0.0831173037221004,
 1: 0.09504599178894553,
 2: 0.21233354753258937,
 3: 0.06190486454238464,
 4: 0.24437164459580676,
 5: 0.14843944968347364,
 6: 0.16898013566739992,
 7: 0.07485891984055598,
 8: 0.16479523366657564,
 9: 0.22126344087807157,
 10: 0.31098300979953336,
 11: 0.22514505950726044,
 12: 0.19423219666087913,
 13: 0.10249426226932923,
 14: 0.0908752085245007,
 15: 0.2054069841322857,
 16: 0.14164525680602222,
 17: 0.05554862672770087,
 18: 0.09728275723533494,
 19: 0.07297345074935374,
 20: 0.05833699329240777,
 21: 0.13984154710230182,
 22: 0.12862022163032077,
 23: 0.08246001401194246,
 24: 0.14191170342543027,
 25: 0.03619772812856633,
 26: 0.13183262604414908,
 27: 0.14500429743421966,
 28: 0.07341209451449741,
 29: 0.23136482572748407,
 30: 0.31098300979953336,
 31: 0.12002453062035272,
 32: 0.17028731157026922,
 33: 0.11703850706479349,
 34: 0.09528339899262096,
 35: 0.10439254696850964,
 36: 0.10109810748864229,
 37: 0.19282100635528263,
 38: 0.0658096903206047,


In [14]:
# vector

[(0, 0.0831173037221004),
 (1, 0.09504599178894553),
 (2, 0.21233354753258937),
 (3, 0.06190486454238464),
 (4, 0.24437164459580676),
 (5, 0.14843944968347364),
 (6, 0.16898013566739992),
 (7, 0.07485891984055598),
 (8, 0.16479523366657564),
 (9, 0.22126344087807157),
 (10, 0.31098300979953336),
 (11, 0.22514505950726044),
 (12, 0.19423219666087913),
 (13, 0.10249426226932923),
 (14, 0.0908752085245007),
 (15, 0.2054069841322857),
 (16, 0.14164525680602222),
 (17, 0.05554862672770087),
 (18, 0.09728275723533494),
 (19, 0.07297345074935374),
 (20, 0.05833699329240777),
 (21, 0.13984154710230182),
 (22, 0.12862022163032077),
 (23, 0.08246001401194246),
 (24, 0.14191170342543027),
 (25, 0.03619772812856633),
 (26, 0.13183262604414908),
 (27, 0.14500429743421966),
 (28, 0.07341209451449741),
 (29, 0.23136482572748407),
 (30, 0.31098300979953336),
 (31, 0.12002453062035272),
 (32, 0.17028731157026922),
 (33, 0.11703850706479349),
 (34, 0.09528339899262096),
 (35, 0.10439254696850964),
 (36,

In [15]:
# len(corpus)

22438

In [ ]:
# for i in range(len(corpus)):
#     vector = model[corpus[0]]

In [24]:
# for corp in corpus:
#     dic = dict(model[corp])
# #     for i in vec:
#     break
# dic

{0: 0.0831173037221004,
 1: 0.09504599178894553,
 2: 0.21233354753258937,
 3: 0.06190486454238464,
 4: 0.24437164459580676,
 5: 0.14843944968347364,
 6: 0.16898013566739992,
 7: 0.07485891984055598,
 8: 0.16479523366657564,
 9: 0.22126344087807157,
 10: 0.31098300979953336,
 11: 0.22514505950726044,
 12: 0.19423219666087913,
 13: 0.10249426226932923,
 14: 0.0908752085245007,
 15: 0.2054069841322857,
 16: 0.14164525680602222,
 17: 0.05554862672770087,
 18: 0.09728275723533494,
 19: 0.07297345074935374,
 20: 0.05833699329240777,
 21: 0.13984154710230182,
 22: 0.12862022163032077,
 23: 0.08246001401194246,
 24: 0.14191170342543027,
 25: 0.03619772812856633,
 26: 0.13183262604414908,
 27: 0.14500429743421966,
 28: 0.07341209451449741,
 29: 0.23136482572748407,
 30: 0.31098300979953336,
 31: 0.12002453062035272,
 32: 0.17028731157026922,
 33: 0.11703850706479349,
 34: 0.09528339899262096,
 35: 0.10439254696850964,
 36: 0.10109810748864229,
 37: 0.19282100635528263,
 38: 0.0658096903206047,


In [ ]:
# vec0 = model[corpus[0]]
# vec1 = model[corpus[1]]

In [17]:
# corpus[-1]

[(3, 1),
 (25, 2),
 (28, 1),
 (106, 1),
 (137, 1),
 (158, 1),
 (188, 3),
 (278, 1),
 (693, 1),
 (698, 1),
 (733, 1),
 (764, 5),
 (834, 1),
 (902, 1),
 (905, 1),
 (921, 1),
 (953, 1),
 (980, 1),
 (999, 4),
 (1030, 1),
 (1083, 1),
 (1118, 1),
 (1210, 1),
 (1215, 1),
 (1243, 1),
 (1280, 1),
 (1370, 1),
 (1371, 1),
 (1460, 1),
 (1470, 2),
 (1485, 1),
 (1488, 1),
 (1498, 1),
 (1499, 2),
 (1506, 1),
 (1512, 2),
 (1526, 1),
 (1583, 1),
 (1645, 3),
 (1986, 3),
 (2080, 1),
 (2248, 1),
 (2365, 1),
 (2678, 1),
 (2841, 1),
 (2920, 1),
 (3361, 1),
 (3450, 1),
 (3543, 1),
 (5807, 1),
 (6153, 1),
 (9808, 1),
 (12796, 1),
 (13203, 1),
 (16858, 1),
 (29569, 8),
 (85710, 1),
 (114437, 1)]

In [21]:
dict(vector)

{0: 0.0831173037221004,
 1: 0.09504599178894553,
 2: 0.21233354753258937,
 3: 0.06190486454238464,
 4: 0.24437164459580676,
 5: 0.14843944968347364,
 6: 0.16898013566739992,
 7: 0.07485891984055598,
 8: 0.16479523366657564,
 9: 0.22126344087807157,
 10: 0.31098300979953336,
 11: 0.22514505950726044,
 12: 0.19423219666087913,
 13: 0.10249426226932923,
 14: 0.0908752085245007,
 15: 0.2054069841322857,
 16: 0.14164525680602222,
 17: 0.05554862672770087,
 18: 0.09728275723533494,
 19: 0.07297345074935374,
 20: 0.05833699329240777,
 21: 0.13984154710230182,
 22: 0.12862022163032077,
 23: 0.08246001401194246,
 24: 0.14191170342543027,
 25: 0.03619772812856633,
 26: 0.13183262604414908,
 27: 0.14500429743421966,
 28: 0.07341209451449741,
 29: 0.23136482572748407,
 30: 0.31098300979953336,
 31: 0.12002453062035272,
 32: 0.17028731157026922,
 33: 0.11703850706479349,
 34: 0.09528339899262096,
 35: 0.10439254696850964,
 36: 0.10109810748864229,
 37: 0.19282100635528263,
 38: 0.0658096903206047,
